In [1]:
import os
os.chdir("/Users/travishong/aml-lsc-hierarchy")
print(os.getcwd())

/Users/travishong/aml-lsc-hierarchy


In [5]:
import scanpy as sc
from src.hierarchy_classifier import load_hierarchy_model, annotate_hierarchy

clf, ref_genes = load_hierarchy_model()

adata = sc.read_h5ad("data/scAML_LSPC_scran.h5ad")

adata_h = annotate_hierarchy(adata, clf, ref_genes, label_key="hierarchy")

In [8]:
from src.lsc_scoring import score_signature

lsc17_genes = [
    "DNMT3B",
    "ZBTB46",
    "NYNRIN",
    "ARHGAP22",
    "LAPTM4B",
    "MMRN1",
    "DPYSL3",
    "KIAA0125",
    "CDK6",
    "CPXM1",
    "SOCS2",
    "SMIM24",
    "EMP1",
    "NGFRAP1",
    "CD34",
    "AKR1C3",
    "GPR56",
]
adata_h = annotate_hierarchy(adata, clf, ref_genes, label_key="hierarchy")
adata_h = score_signature(adata_h, lsc17_genes, "LSC17_score")

prim = adata_h[adata_h.obs["hierarchy"] == "Primitive"]
thr = prim.obs["LSC17_score"].quantile(0.9)

adata_h.obs["LSC_high"] = (
    (adata_h.obs["hierarchy"] == "Primitive") &
    (adata_h.obs["LSC17_score"] >= thr)
)

set(g for g in lsc17_genes if g in adata_h.var_names)


{'AKR1C3', 'CDK6', 'CPXM1', 'SOCS2'}

In [10]:
prim = adata_h[adata_h.obs["hierarchy"] == "Primitive"]
thr = prim.obs["LSC17_score"].quantile(0.9)

adata_h.obs["LSC_high"] = (
    (adata_h.obs["hierarchy"] == "Primitive") &
    (adata_h.obs["LSC17_score"] >= thr)
)

In [11]:
prim = adata_h[adata_h.obs["hierarchy"] == "Primitive"].copy()

thr = prim.obs["LSC17_score"].quantile(0.9)

adata_h.obs["LSC_high"] = (
    (adata_h.obs["hierarchy"] == "Primitive") &
    (adata_h.obs["LSC17_score"] >= thr)
)

print("Total cells:", adata_h.n_obs)
print("Primitive cells:", (adata_h.obs['hierarchy'] == 'Primitive').sum())
print("LSC_high cells:", adata_h.obs["LSC_high"].sum())

Total cells: 1948
Primitive cells: 604
LSC_high cells: 61


In [12]:
import pandas as pd

print(pd.crosstab(adata_h.obs["hierarchy"], adata_h.obs["LSC_high"]))

LSC_high    False  True 
hierarchy               
Primitive     543     61
Progenitor   1344      0
